# CODICE PER USARE OPEN STREET MAP

In [ ]:
#LIBRARIES NEEDED 
import pandas as pd
import osmnx as ox

#per plottare
import geopandas as gpd
from shapely import wkt
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
# CODICE COMPLETO PER AMENITIES NEL RAGGIO DI 800M

FILE_PATH = 'PATH TO YOUR/pdr_processed.csv' 
df = pd.read_csv(FILE_PATH)

# Extract unique latitude and longitude pairs and Convert to a list of tuples
unique_coords = df[['station_coordinates_lat', 'station_coordinates_lon']].drop_duplicates()
unique_coords_list = list(unique_coords.itertuples(index=False, name=None))

# Create an empty DataFrame to store all POI data
all_pois = pd.DataFrame()

# Function to extract POIs around each charging point
def extract_pois(lat, lon, radius=800):
    # Define the tags for relevant amenities
    tags = {
        'amenity': [
            'restaurant', 'cafe', 'fast_food', 'pub', 'bar', 'food_court',  # Food & dining
            'gym',  # Fitness centers
            'supermarket', 'grocery',  # Shopping & groceries
            'college', 'research_institute', 'university',  # Educational facilities
            'office', 'coworking_space',  # Workplaces
            'court_house', 'post_office',  # Public services
            'cinema', 'conference_centre', 'events_venue', 'exhibition_centre', 'theatre'  # Arts and entertainment
        ],
        'shop': ['mall', 'retail', 'department_store'],  # Shopping centers
    }
    
    try:
        # Download POIs within the radius using osmnx
        poi_data = ox.geometries_from_point((lat, lon), tags=tags, dist=radius)
        
        # Filter to keep only points and exclude polygons or multipolygons
        poi_data = poi_data[poi_data.geometry.type == 'Point']
        
        # If no points are found, return an empty DataFrame
        if poi_data.empty:
            return pd.DataFrame()

        # Convert to DataFrame and add latitude and longitude columns from geometry
        poi_df = pd.DataFrame(poi_data)
        poi_df['latitude'] = poi_data.geometry.y
        poi_df['longitude'] = poi_data.geometry.x
        
        # Return the filtered POI DataFrame
        return poi_df
    
    except Exception as e:
        print(f"Error fetching POIs for location ({lat}, {lon}): {e}")
        return pd.DataFrame()  # Return an empty DataFrame if an error occurs

# Loop through the list of unique coordinates and extract POIs
for lat, lon in unique_coords_list:
    pois = extract_pois(lat, lon)
    all_pois = pd.concat([all_pois, pois], ignore_index=True)  # Append to the main DataFrame

# Save the combined POI data to a CSV file
output_file_path = 'extracted_pois.csv'
all_pois.to_csv(output_file_path, index=False)
print(f"POI data has been saved to {output_file_path}")

In [ ]:
FILE_PATH = 'PATH TO YOUR/extracted_pois.csv'
extracted_df = pd.read_csv('FILE_PATH')
extracted_df = extracted_df[['amenity', 'geometry', 'name']]
extracted_df.to_csv(FILE_PATH, index=False)

In [ ]:
'''#PLOT AMENITIES COUNTER 

 Questo codice va cambiato perché bisogna 
 richiamare il df con in count delle 
 amenities per poi fare il plot

# Define colors inspired by the Free To X logo
colors = ['#0071BC', '#FFD100', '#4D4D4F']  # Blue, yellow, gray

# Plot
plt.figure(figsize=(12, 6))
bars = plt.bar(df['amenity'], df['count'], color=colors[0])  # Using blue as the primary color

# Customize each bar color based on index for variety (using a repeating color pattern)
for i, bar in enumerate(bars):
    bar.set_color(colors[i % len(colors)])

# Title and labels
plt.title('Count of Amenities Near Charging Stations', fontsize=16, weight='bold')
plt.xlabel('Amenity Type', fontsize=14)
plt.ylabel('Count', fontsize=14)

# Rotate x-axis ticks for readability
plt.xticks(rotation=45, ha='right', fontsize=12)

# Customize y-axis ticks and grid
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Display the plot
plt.tight_layout()
plt.show()
'''

In [ ]:
# QUESTO È PER PLOTTARE

# Convert the 'geometry' column from WKT format to actual geometry using shapely
extracted_df['geometry'] = extracted_df['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(extracted_df, geometry='geometry')

# Filter only POINT geometries for mapping, as polygons have multiple coordinates
point_data = gdf[gdf['geometry'].type == 'Point'].copy()

# Extract latitude and longitude from the POINT geometry
point_data['latitude'] = point_data['geometry'].y
point_data['longitude'] = point_data['geometry'].x

# Plot using plotly express
fig = px.scatter_mapbox(
    point_data,
    lat='latitude',
    lon='longitude',
    color='amenity',
    hover_name='amenity',
    zoom=6,
    mapbox_style='open-street-map',
    title='Visualizzazione dei Punti di Interesse in Italia'
)

# Show the map
fig.show()

# JUST FOR TOP 20 NOW

In [2]:
import pandas as pd

In [ ]:
dfric = pd.read_csv("PATH TO/top20ricavi.csv")
dfpami = pd.read_csv("PATH TO/extracted_pois.csv")

In [ ]:
# ordering dfric by ricavi
dfric = dfric.sort_values(by='Ricavi totali (€) (IVA esclusa)', ascending=False)
# keeping only first 20 rows
dfric = dfric[:20]

In [ ]:
# AMENITIES VICINO TOP 20 per ricavi

# Extract unique latitude and longitude pairs
unique_coords = dfric[['station_coordinates_lat', 'station_coordinates_lon']].drop_duplicates()

# Convert to a list of tuples
unique_coords_list = list(unique_coords.itertuples(index=False, name=None))

print(unique_coords_list)

# Create an empty DataFrame to store all POI data
all_pois = pd.DataFrame()

# Function to extract POIs around each charging point
def extract_pois(lat, lon, radius=800):
    # Define the tags for relevant amenities
    tags = {
        'amenity': [
            'restaurant', 'cafe', 'fast_food', 'pub', 'bar', 'food_court',  # Food & dining
            'gym',  # Fitness centers
            'supermarket', 'grocery',  # Shopping & groceries
            'college', 'research_institute', 'university',  # Educational facilities
            'office', 'coworking_space',  # Workplaces
            'court_house', 'post_office',  # Public services
            'cinema', 'conference_centre', 'events_venue', 'exhibition_centre', 'theatre'  # Arts and entertainment
        ],
        'shop': ['mall', 'retail', 'department_store'],  # Shopping centers
    }
    
    try:
        # Download POIs within the radius using osmnx
        poi_data = ox.geometries_from_point((lat, lon), tags=tags, dist=radius)
        
        # Filter to keep only points and exclude polygons or multipolygons
        poi_data = poi_data[poi_data.geometry.type == 'Point']
        
        # If no points are found, return an empty DataFrame
        if poi_data.empty:
            return pd.DataFrame()

        # Convert to DataFrame and add latitude and longitude columns from geometry
        poi_df = pd.DataFrame(poi_data)
        poi_df['latitude'] = poi_data.geometry.y
        poi_df['longitude'] = poi_data.geometry.x
        
        # Return the filtered POI DataFrame
        return poi_df
    
    except Exception as e:
        print(f"Error fetching POIs for location ({lat}, {lon}): {e}")
        return pd.DataFrame()  # Return an empty DataFrame if an error occurs

# Loop through the list of unique coordinates and extract POIs
for lat, lon in unique_coords_list:
    pois = extract_pois(lat, lon)
    all_pois = pd.concat([all_pois, pois], ignore_index=True)  # Append to the main DataFrame

# Save the combined POI data to a CSV file
output_file_path = 'Top20amenities.csv'
all_pois.to_csv(output_file_path, index=False)
print(f"POI data has been saved to {output_file_path}")

In [11]:
dfamenities = pd.read_csv('/Users/paoloemiliogrande/Desktop/BIG DATA/Top20amenities.csv')

In [12]:
dfamenities.shape

(97, 72)

In [13]:
dfamenities = dfamenities[['amenity', 'geometry', 'name']]
dfamenities.head()

,amenity,geometry,name
0,cafe,POINT (10.8558537 44.6617058),Eataly
1,cafe,POINT (10.8578243 44.6625963),Acafé
2,fast_food,POINT (10.8576345 44.6627165),Burger King
3,restaurant,POINT (10.8574166 44.6625261),Sarni
4,restaurant,POINT (10.8557309 44.6618607),Autogrill


In [14]:
import pandas as pd
from shapely.geometry import Point

# Convert 'station_coordinates_lat' and 'station_coordinates_lon' to 'geometry' column in POINT format
dfric['geometry'] = dfric.apply(
    lambda row: Point(row['station_coordinates_lon'], row['station_coordinates_lat']),
    axis=1
)

# Drop the original latitude and longitude columns if desired
# chargers_df = chargers_df.drop(columns=['station_coordinates_lat', 'station_coordinates_lon'])

# Display the resulting DataFrame
print(dfric[['station_coordinates_lat', 'station_coordinates_lon', 'geometry']].head())

# Save to a new CSV if needed
dfric.to_csv('dfric_with_geometry.csv', index=False)
print("Chargers data with geometry in POINT format has been saved to 'chargers_with_geometry.csv'")


     station_coordinates_lat  station_coordinates_lon  \
128                44.662303                10.858004   
419                44.962798                 9.906515   
416                44.962843                 9.906473   
418                44.962798                 9.906515   
130                44.662299                10.857972   

                        geometry  
128  POINT (10.858004 44.662303)  
419   POINT (9.906515 44.962798)  
416   POINT (9.906473 44.962843)  
418   POINT (9.906515 44.962798)  
130  POINT (10.857972 44.662299)  
Chargers data with geometry in POINT format has been saved to 'chargers_with_geometry.csv'


In [15]:
dfric2 = pd.read_csv("/Users/paoloemiliogrande/Desktop/BIG DATA/dfric_with_geometry.csv")
dfric2.head()

,connector_evse_id,station_name,station_address,station_postal_code,station_coordinates_lat,station_coordinates_lon,station_brand,totEnergy,totSessions,totCost,Potenza (kW),Station Città,Ricavi totali (€) (IVA esclusa),Tempo Totale (min),denominazione_regione,EV_percentage_per_region,geometry
0,IT*F2X*EF2XITA3001*1,Area di Servizio - Secchia Est,Autostrada del Sole,41100,44.662303,10.858004,alpitronic GmbH,54080.240,1620,36771.63,300.0,Modena,59523.37,72659.0,Emilia-Romagna,0.0661,POINT (10.858004 44.662303)
1,IT*F2X*EF2XITA2002*2,Area di Servizio - Arda Est,A1 - Autostrada del Sole,29017,44.962798,9.906515,alpitronic GmbH,74152.352,2092,50355.66,300.0,Casa Nuova,54209.87,62399.0,Emilia-Romagna,0.0661,POINT (9.906515 44.962798)
2,IT*F2X*EF2XITA2001*1,Area di Servizio - Arda Est,A1 - Autostrada del Sole,29017,44.962843,9.906473,alpitronic GmbH,68450.144,2136,46524.74,300.0,Casa Nuova,51286.73,57684.0,Emilia-Romagna,0.0661,POINT (9.906473 44.962843)
3,IT*F2X*EF2XITA2002*1,Area di Servizio - Arda Est,A1 - Autostrada del Sole,29017,44.962798,9.906515,alpitronic GmbH,68289.672,2175,46492.95,300.0,Casa Nuova,51205.17,58379.0,Emilia-Romagna,0.0661,POINT (9.906515 44.962798)
4,IT*F2X*EF2XITA3002*1,Area di Servizio - Secchia Est,Autostrada del Sole,41100,44.662299,10.857972,alpitronic GmbH,56488.656,1694,38425.58,300.0,Modena,50395.54,60386.0,Emilia-Romagna,0.0661,POINT (10.857972 44.662299)


In [16]:
# just keep geometry column
dfric2 = dfric2[['geometry']]
dfric2.head()

,geometry
0,POINT (10.858004 44.662303)
1,POINT (9.906515 44.962798)
2,POINT (9.906473 44.962843)
3,POINT (9.906515 44.962798)
4,POINT (10.857972 44.662299)


In [17]:
# add a column named 'amenity' and fill all with 'FTX charger'

dfric2['amenity'] = 'FTX charger'
dfric2.head()

,geometry,amenity
0,POINT (10.858004 44.662303),FTX charger
1,POINT (9.906515 44.962798),FTX charger
2,POINT (9.906473 44.962843),FTX charger
3,POINT (9.906515 44.962798),FTX charger
4,POINT (10.857972 44.662299),FTX charger


In [18]:
# merge the two dataframes
dfmerged = pd.concat([dfamenities, dfric2], ignore_index=True)
dfmerged.head()

,amenity,geometry,name
0,cafe,POINT (10.8558537 44.6617058),Eataly
1,cafe,POINT (10.8578243 44.6625963),Acafé
2,fast_food,POINT (10.8576345 44.6627165),Burger King
3,restaurant,POINT (10.8574166 44.6625261),Sarni
4,restaurant,POINT (10.8557309 44.6618607),Autogrill


In [19]:
dfmerged.sample(20)

,amenity,geometry,name
70,cafe,POINT (9.2804437 45.4669398),L'Orchidea
82,bar,POINT (10.1578575 45.5254892),Nuovo Bar Aki
7,restaurant,POINT (9.9082643 44.9626295),Sarni
10,fast_food,POINT (10.8576345 44.6627165),Burger King
80,bar,POINT (10.1581173 45.5256068),Flower Cafè
60,cafe,POINT (8.8102198 45.7557623),La Papaya
9,cafe,POINT (10.8578243 44.6625963),Acafé
52,post_office,POINT (9.2797801 45.4611451),NaN
33,restaurant,POINT (10.1488142 45.5321101),Autogrill
24,fast_food,POINT (10.1569307 45.5255191),Pizzeria la Verace


In [21]:
# let's plot the map
import pandas as pd
import geopandas as gpd
from shapely import wkt
import plotly.express as px

# Load data with pandas
data = dfmerged

# Convert the 'geometry' column from WKT format to actual geometry using shapely
data['geometry'] = data['geometry'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry')

# Filter only POINT geometries for mapping, as polygons have multiple coordinates
point_data = gdf[gdf['geometry'].type == 'Point'].copy()

# Extract latitude and longitude from the POINT geometry
point_data['latitude'] = point_data['geometry'].y
point_data['longitude'] = point_data['geometry'].x

# Plot using plotly express
fig = px.scatter_mapbox(
    point_data,
    lat='latitude',
    lon='longitude',
    color='amenity',
    hover_name='amenity',
    zoom=6,
    mapbox_style='open-street-map',
    title='Visualizzazione dei Punti di Interesse in Italia'
)

# Show the map
fig.show()

In [24]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load your dataset (assuming it contains both chargers and amenities)
data = dfmerged

# Function to safely convert geometry column to Point
def convert_to_point(geometry):
    if isinstance(geometry, str):  # Check if it's a string
        # Strip "POINT" and parse coordinates, then create a Point
        coordinates = geometry.replace("POINT ", "").strip("()").split()
        return Point(float(coordinates[0]), float(coordinates[1]))
    elif isinstance(geometry, Point):  # If already a Point object
        return geometry
    else:
        raise ValueError("Unexpected geometry format")

# Apply the function to convert the 'geometry' column
data['geometry'] = data['geometry'].apply(convert_to_point)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(data, geometry='geometry', crs="EPSG:4326")

# Separate chargers and amenities
chargers = gdf[gdf['amenity'] == 'FTX charger'].copy()  # Assuming 'FTX charger' is the label for chargers
amenities = gdf[gdf['amenity'] != 'FTX charger'].copy()  # Everything else is considered an amenity

# Convert to a projected CRS (meters) to accurately calculate distances
chargers = chargers.to_crs(epsg=3857)
amenities = amenities.to_crs(epsg=3857)

# Function to find amenities within 800m for each charger and list unique types
def find_amenities_around_charger(charger, amenities, radius=800):
    # Calculate distances to amenities and filter within the radius
    nearby_amenities = amenities[amenities.geometry.distance(charger.geometry) <= radius]
    # Get unique amenity types
    unique_amenities = nearby_amenities['amenity'].unique().tolist()
    return unique_amenities

# Apply the function to each charger
chargers['nearby_amenities'] = chargers.apply(lambda charger: find_amenities_around_charger(charger, amenities), axis=1)

# Save the result to a new CSV file
output_file_path = 'chargers_with_nearby_amenities.csv'
chargers.to_csv(output_file_path, index=False)
print(f"Chargers with nearby amenities have been saved to {output_file_path}")


Chargers with nearby amenities have been saved to chargers_with_nearby_amenities.csv


In [26]:
prova = pd.read_csv('/Users/paoloemiliogrande/Desktop/BIG DATA/chargers_with_nearby_amenities.csv')
prova.sample(20)

,amenity,geometry,name,nearby_amenities
12,FTX charger,POINT (1045658.4860633644 5712351.211420403),NaN,[]
6,FTX charger,POINT (1208707.4763113274 5568513.757297508),NaN,"['cafe', 'fast_food', 'restaurant']"
5,FTX charger,POINT (1208703.9140876222 5568513.1312583275),NaN,"['cafe', 'fast_food', 'restaurant']"
18,FTX charger,POINT (1523315.1702870626 5086037.596075952),NaN,['restaurant']
16,FTX charger,POINT (1130024.633828826 5705757.523794186),NaN,"['cafe', 'restaurant', 'bar', 'pub', 'fast_food']"
10,FTX charger,POINT (1102783.5299173133 5615673.772552241),NaN,['restaurant']
8,FTX charger,POINT (1102556.1041976225 5615637.11531034),NaN,['restaurant']
14,FTX charger,POINT (980821.5618457302 5742355.862570743),NaN,['restaurant']
13,FTX charger,POINT (1032640.0055735636 5694526.694155467),NaN,"['cafe', 'restaurant', 'fast_food', 'post_offi..."
17,FTX charger,POINT (1468150.3513464928 5114137.608482699),NaN,[]


In [27]:
import pandas as pd

# Load your data
data = prova

# Convert the 'nearby_amenities' column from string to list (if it's not already a list)
data['nearby_amenities'] = data['nearby_amenities'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Get a list of all unique amenities across all chargers
all_amenities = set(amenity for sublist in data['nearby_amenities'] for amenity in sublist)

# Create a boolean column for each unique amenity type
for amenity in all_amenities:
    data[amenity] = data['nearby_amenities'].apply(lambda x: amenity in x)

# Drop 'nearby_amenities' column if you no longer need it
# data = data.drop(columns=['nearby_amenities'])

# Display the updated DataFrame
print(data.head())

# Save the result to a new CSV file if needed
output_file_path = '/Users/paoloemiliogrande/Desktop/BIG DATA/chargers_with_all_amenities_booleans.csv'
data.to_csv(output_file_path, index=False)
print(f"Chargers with all amenities booleans have been saved to {output_file_path}")


       amenity                                       geometry  name  \
0  FTX charger   POINT (1208707.4763113274 5568513.757297508)   NaN   
1  FTX charger  POINT (1102788.2053359265 5615666.6928158095)   NaN   
2  FTX charger   POINT (1102783.5299173133 5615673.772552241)   NaN   
3  FTX charger  POINT (1102788.2053359265 5615666.6928158095)   NaN   
4  FTX charger  POINT (1208703.9140876222 5568513.1312583275)   NaN   

                nearby_amenities    pub  fast_food  restaurant  post_office  \
0  [cafe, fast_food, restaurant]  False       True        True        False   
1                   [restaurant]  False      False        True        False   
2                   [restaurant]  False      False        True        False   
3                   [restaurant]  False      False        True        False   
4  [cafe, fast_food, restaurant]  False       True        True        False   

    cafe    bar  
0   True  False  
1  False  False  
2  False  False  
3  False  False  
4   True

In [28]:
prova2 = pd.read_csv('/Users/paoloemiliogrande/Desktop/BIG DATA/chargers_with_all_amenities_booleans.csv')
prova2.sample(20)

,amenity,geometry,name,nearby_amenities,pub,fast_food,restaurant,post_office,cafe,bar
9,FTX charger,POINT (1208351.6992187523 5568406.0791937),NaN,"['cafe', 'fast_food', 'restaurant']",False,True,True,False,True,False
18,FTX charger,POINT (1523315.1702870626 5086037.596075952),NaN,['restaurant'],False,False,True,False,False,False
1,FTX charger,POINT (1102788.2053359265 5615666.6928158095),NaN,['restaurant'],False,False,True,False,False,False
7,FTX charger,POINT (1129716.0562003471 5705748.624642473),NaN,"['restaurant', 'bar', 'pub', 'fast_food']",True,True,True,False,False,True
2,FTX charger,POINT (1102783.5299173133 5615673.772552241),NaN,['restaurant'],False,False,True,False,False,False
16,FTX charger,POINT (1130024.633828826 5705757.523794186),NaN,"['cafe', 'restaurant', 'bar', 'pub', 'fast_food']",True,True,True,False,True,True
10,FTX charger,POINT (1102783.5299173133 5615673.772552241),NaN,['restaurant'],False,False,True,False,False,False
14,FTX charger,POINT (980821.5618457302 5742355.862570743),NaN,['restaurant'],False,False,True,False,False,False
0,FTX charger,POINT (1208707.4763113274 5568513.757297508),NaN,"['cafe', 'fast_food', 'restaurant']",False,True,True,False,True,False
12,FTX charger,POINT (1045658.4860633644 5712351.211420403),NaN,[],False,False,False,False,False,False
